In [3]:
import os

import torch
from torch.nn.utils.rnn import pack_sequence, unpack_sequence, pad_sequence
import h5py as h5
import numpy as np
import polars as pl

from root_manager.chunk_generator import ChunkGenerator
from root_manager.settings import ChunkGeneratorConfig, ProcessorConfig, FilterParams

In [4]:
path_mu = "/net/62/home3/ivkhar/Baikal/data/initial_data/MC_2020/muatm/root/all/"
path_nuatm = "/net/62/home3/ivkhar/Baikal/data/initial_data/MC_2020/nuatm/root/all/"
path_nu2 = "/net/62/home3/ivkhar/Baikal/data/initial_data/MC_2020/nue2_100pev/root/all/"
def explore_paths(p: str, start: int, stop: int):
    files = sorted(os.listdir(p))[start:stop]
    return [f"{p}{file}" for file in files]
mu_paths = explore_paths(path_mu, 0, 25)

In [5]:
proc_cfg = ProcessorConfig(
    center_times= True, 
    calc_tres = False, 
    filter_cfg = FilterParams(
        only_signal = False,
        min_hits = 0,
        min_strings = 0,
        min_Q = 0,
        t_threshold = 100000
    )
)
chunk_generator_cfg=ChunkGeneratorConfig(
            chunk_size = 25,
            processor_params = proc_cfg,
            fields = ['PulsesAmpl', 'PulsesTime', 'Xrel', 'Yrel', 'Zrel', 'num_signal_hits', 'num_signal_strings', 'nu_induced'],
            shuffle_paths = False
        )

gen = ChunkGenerator(mu_paths, chunk_generator_cfg).get_chunks()
df = next(gen)
df_features = df[['PulsesAmpl', 'PulsesTime', 'Xrel', 'Yrel', 'Zrel']]

data = df_features.to_numpy()[:,:5]
for i in range(len(data)):
    data[i] = data[i, :]
    for j in range(len(data[i])):
        data[i, j] = data[i, j].round(3).astype(np.float32)

In [6]:
# Convert the list of numpy arrays to torch tensors
torch_tensors = list(map(torch.from_numpy, map(lambda x: np.stack(x, axis=-1), list(df_features.to_numpy()))))
#torch_tensors = pad_sequence(torch_tensors, batch_first=True, padding_value=float('nan'))
# data_seq, _, sorted_idxs, idxs = pack_sequence(torch_tensors, enforce_sorted=False)
# t = pack_sequence(torch_tensors, enforce_sorted=False)
# unpack_sequence(t)[0]

In [9]:
for arr in map(lambda x: np.stack(x, axis=-1), list(df_features.to_numpy())):
    torch.from_numpy(arr)

In [121]:
torch_tensors.shape

torch.Size([32680, 148, 5])

In [68]:
pack_sequence(df_features['PulsesAmpl'].to_list(), enforce_sorted = False)

AttributeError: 'list' object has no attribute 'size'

In [46]:
pl_data

data
struct[5]
"{[1.910719, 0.680661, … 0.79696],[-2538.06543, -2533.398438, … 2374.409668],[-58.110868, -40.877882, … -31.008856],[-4.861609, 41.557367, … -49.470633],[247.525992, 7.690028, … 7.319012]}"
"{[0.630403, 0.819006, … 1.506786],[-2728.459961, -2557.720947, … 2251.506592],[53.961874, -40.992133, … -0.395121],[-19.536512, 42.604509, … -1.683516],[247.731508, 127.881517, … -7.559459]}"
"{[1.24319, 0.45773, … 2.384422],[-2401.945068, -2352.771484, … 2511.410889],[-53.664543, 60.158501, … -8.936485],[27.840374, -2.030605, … 57.560368],[203.033434, 202.674502, … 218.029498]}"
"{[0.330916, 0.959203, … 1.128354],[-2343.806396, -2207.74707, … 2409.185547],[17.954746, 62.309757, … -62.23724],[-55.696244, -15.339253, … -12.04324],[-83.579933, 262.120064, … 7.328074]}"
"{[1.19767, 1.896235, … 1.27869],[-2306.198486, -2256.801758, … 2628.382324],[2.177528, 34.699516, … -55.225517],[-0.764629, -46.808651, … -23.255611],[-263.652494, 217.675479, … 233.055514]}"
…
"{[1.045604, 0.780759, … 0.657507],[-2200.046875, -2197.669922, … 2780.611328],[-33.253271, -41.625219, … -58.665289],[-49.427752, 44.301244, … -4.282756],[-233.135579, -97.345578, … 172.90942]}"
"{[0.343402, 0.499559, … 0.479027],[-2306.005859, -2245.419434, … 2608.529785],[-40.992133, -40.992133, … 4.32488],[42.604509, 42.604509, … 58.976503],[-67.361471, 127.881517, … 172.776476]}"
"{[0.189371, 1.033087, … 1.485312],[-2215.638672, -2204.812988, … 2600.888916],[3.311741, -62.23724, … 62.309757],[-0.165249, -12.04324, … -15.339253],[132.839088, -203.086922, … 111.798073]}"


In [4]:
import numpy as np
import torch
from torch.nn.utils.rnn import pack_sequence

def list_to_packed_sequence(array_list):
    # Convert each numpy array in the list to a torch tensor
    tensor_list = [torch.tensor(arr, dtype=torch.float32) for arr in array_list]
    
    # Pack the sequence directly, this will handle variable lengths efficiently
    packed_sequence = pack_sequence(tensor_list, enforce_sorted=False)
    
    return packed_sequence


In [8]:
t = list_to_packed_sequence(list(df_features.to_numpy()))

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [14]:
np.concatenate(data, axis=1).shape

AxisError: axis 1 is out of bounds for array of dimension 1

In [15]:
data.shape

(32680, 5)